# Useful links

- [ ] https://cran.r-project.org/web/packages/afex/vignettes/introduction-mixed-models.pdf
- [ ] https://cran.r-project.org/web/packages/afex/afex.pdf
- [ ] https://cran.r-project.org/web/packages/afex/index.html
- [ ] https://cran.r-project.org/web/packages/afex/vignettes/afex_analysing_accuracy_data.html
- [ ] https://cran.r-project.org/web/packages/afex/vignettes/afex_anova_example.html
- [ ] https://cran.r-project.org/web/packages/afex/vignettes/afex_mixed_example.html
- [ ] https://cran.r-project.org/web/packages/emmeans/vignettes/comparisons.html
- [ ] https://cran.r-project.org/web/packages/emmeans/vignettes/messy-data.html
- [ ] https://cran.r-project.org/web/packages/emmeans/vignettes/interactions.html
- [ ] https://cran.r-project.org/web/packages/emmeans/vignettes/basics.html
- [ ] https://cran.r-project.org/web/packages/afex/vignettes/afex_mixed_example.html

- [ ] https://osf.io/nfwx7/

- [ ] https://stats.stackexchange.com/questions/13166/rs-lmer-cheat-sheet

- [ ] https://psu-psychology.github.io/r-bootcamp-2018/index.html
- [ ] http://tysonbarrett.com/EDUC-6600/Resources/Example_Ch15_rmANOVA.html
- [ ] http://singmann.org/anova-in-r-afex-may-be-the-solution-you-are-looking-for/
- [ ] https://ademos.people.uic.edu/Chapter21.html
- [ ] https://www.psychologie.uni-heidelberg.de/ae/meth/team/mertens/blog/anova_in_r_made_easy.nb.html
- [ ] http://tysonbarrett.com/EDUC-6600/Resources/Example_Ch15_rmANOVA.html

# Setup

## Imports

In [7]:
library("dplyr")
library("tidyr")
library("afex")     # provides mixed() and attaches lme4 automatically.
library("emmeans")  # follow-up tests.
library("multcomp") # advanced control for multiple testing/Type 1 errors.
library("ggplot2")

In [8]:
library("car")
require("lattice")
require("ez")

In [9]:
library("cowplot")
library("ggbeeswarm")

library("nortest")

In [10]:
# library("apa")

In [11]:
library("effectsize")

## Settings

In [12]:
theme_set(theme_bw(base_size = 15) +
            theme(legend.position="bottom",
                  panel.grid.major.x = element_blank()))

In [13]:
options(width=120)

## Session info

In [14]:
sessionInfo()

R version 3.6.3 (2020-02-29)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 18.04.5 LTS

Matrix products: default
BLAS:   /usr/lib/x86_64-linux-gnu/openblas/libblas.so.3
LAPACK: /usr/lib/x86_64-linux-gnu/libopenblasp-r0.2.20.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C               LC_TIME=en_IL.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_IL.UTF-8    LC_MESSAGES=en_US.UTF-8    LC_PAPER=en_IL.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C             LC_MEASUREMENT=en_IL.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] effectsize_0.3.3 nortest_1.0-4    ggbeeswarm_0.6.0 cowplot_1.1.0    ez_4.4-0         lattice_0.20-41 
 [7] car_3.0-10       carData_3.0-4    ggplot2_3.3.2    multcomp_1.4-14  TH.data_1.0-10   MASS_7.3-53     
[13] survival_3.2-7   mvtnorm_1.1-1    emmeans_1.5.1    afex_0.28-0    

## Create output directory

In [15]:
dir.create("data/t1007",showWarnings = FALSE,recursive = TRUE)

## Handy functions

### Function for logging information BEFORE execution

In [16]:
logging0 <- function(ifName,infoStr="",data){
    t0 <- Sys.time()
    s0 <- gsub("\\s","_",t0)
    s1 <- gsub("^.","",tempfile("","",""))
    s2 <- gsub("/t1006/","/t1007/",ifName)
    ofName <- paste(paste(s2,"",s0,"",s1,infoStr,"_nrow",nrow(data),sep="_"),".RData",sep="")
    lfName = paste(ofName ,".LOG.txt" ,sep="" ,collapse = "")
    log_msg <- toString(paste(
        paste("logs0" ,toString(lfName)     ,sep=": " ,collapse = "")
      , paste("time0" ,toString(Sys.time()) ,sep=": " ,collapse = "")
      , paste("nrows" ,toString(nrow(data))  ,sep=": " ,collapse = "")
      , paste("ncols" ,toString(ncol(data))  ,sep=": " ,collapse = "")
      , paste("vList" ,toString(ls())       ,sep=": " ,collapse = "")
      , paste("oSave" ,toString(ofName)     ,sep=": " ,collapse = "")        
      , "\n"
      , sep="\n"))
    cat(log_msg)
    write(paste(log_msg) ,file=lfName ,append=TRUE)
    return(list("ofName"=ofName,"lfName"=lfName,"t0"=t0))
}

### Function for logging information AFTER execution

In [17]:
logging1 <- function(lfName,t0){
    t1 <- Sys.time()
    d1 <- t1-t0
    log_msg <- toString(paste(
        paste("time1" ,toString(Sys.time()) ,sep=": " ,collapse = "")
      , paste("diff1" ,capture.output(d1)   ,sep=": " ,collapse = "")
      , "\n"
      , sep="\n"))
    cat(log_msg)
    write(paste(log_msg) ,file=lfName ,append=TRUE)
    return(list("d1" = d1))
}

### Function for displaying results in APA 6

In [18]:
library(xtable)
library(IRdisplay)
library(repr)
require("papaja")
require("tinytex")
apaMd <- function(apaObj){
    apaStr <- ""
    for (ii in names(apaObj)){
        apaStr <- paste(apaStr, paste("\n","### ", ii, "\n" ,sep="" ,collapse = "")  ,sep="" ,collapse = "")
        if(!(ii == "table")){ 
            for (jj in names(apaObj[[ii]])){
                apaStr <- paste(apaStr, paste("- ", ii," for `", jj, "` is ", "\n" ,sep="" ,collapse = "")  ,sep="\n" ,collapse = " ")
                apaStr <- paste(apaStr, paste("  - ", apaObj[[ii]][[jj]], "\n" ,sep="" ,collapse = "")  ,sep="\n" ,collapse = " ")
                }}
        else {
            # apaStr <- paste(apaStr, "\n\n", paste(apa_table(apaObj$table))  ,sep="\n" ,collapse = " ")
            apaStr <- paste(apaStr, paste("- support for will arrive some day", "\n" ,sep="" ,collapse = "")  ,sep="\n" ,collapse = " ")
            apaStr <- paste(apaStr, paste("- for now please use apa_table function", "\n" ,sep="" ,collapse = "")  ,sep="\n" ,collapse = " ")

        }}
    return(apaStr)
}


Attaching package: ‘IRdisplay’


The following object is masked from ‘package:xtable’:

    display


Loading required package: papaja

Loading required package: tinytex



# Load data

Depending on how much data you want to include select a file to be loaded (either by executing relevant cell or by clicking ↑ or ↓ above to move cells up or down, the last executed cell containing `ifName` defines the file to be loaded).

In [19]:
ifName="data/t1006/bigDF__df2__001_unstacked.RData" # 752554 obs

In [20]:
load(file=ifName)

In [21]:
df3 <- df2

In [22]:
df2 <- subset(df2,wordCnt>4) # ~649262 obs

In [66]:
df2$LBL <- paste(df2$CAMP,"_",df2$TYPE, sep="")

# Inspect data

## Quick look at data

In [67]:
str(df2)

'data.frame':	681652 obs. of  21 variables:
 $ CAMP       : Factor w/ 2 levels "metoo","sexstrike": 1 1 1 1 1 1 1 1 1 1 ...
 $ TYPE       : Factor w/ 2 levels "ctrl","orig": 1 1 1 1 1 1 1 1 1 1 ...
 $ wordCnt    : num  11 7 11 27 30 26 8 12 12 23 ...
 $ vb01Cnt    : num  0 0 0 2 0 0 1 0 1 0 ...
 $ nn01Cnt    : num  1 2 0 2 3 4 1 4 1 4 ...
 $ ag01Cnt    : num  0 0 0 0 1 1 0 0 1 2 ...
 $ ag02Cnt    : num  0 0 0 0 1 0 0 0 1 0 ...
 $ vb01Prc    : num  0 0 0 0.0741 0 ...
 $ nn01Prc    : num  0.0909 0.2857 0 0.0741 0.1 ...
 $ ag01Prc    : num  0 0 0 0 0.0333 ...
 $ ag02Prc    : num  0 0 0 0 0.0333 ...
 $ ID         : chr  "919714354467045376" "919714354479722496" "919714354500653056" "919714354542600192" ...
 $ tweet_url  : chr  "/IamJonesy3/status/919714354467045376" "/CCmee_/status/919714354479722496" "/SarahJaneVlogs/status/919714354500653057" "/queentyreen/status/919714354542600195" ...
 $ user_id    : chr  "588400632" "242582835" "29298715" "3290716455" ...
 $ is_replied : logi  TRUE FA

## Summary

In [24]:
summary(df2)

        CAMP          TYPE           wordCnt          vb01Cnt           nn01Cnt          ag01Cnt       
 metoo    :613635   ctrl:353731   Min.   :  5.00   Min.   : 0.0000   Min.   : 0.000   Min.   : 0.0000  
 sexstrike: 68017   orig:327921   1st Qu.: 11.00   1st Qu.: 0.0000   1st Qu.: 2.000   1st Qu.: 0.0000  
                                  Median : 18.00   Median : 1.0000   Median : 3.000   Median : 0.0000  
                                  Mean   : 18.43   Mean   : 0.8063   Mean   : 3.767   Mean   : 0.6863  
                                  3rd Qu.: 24.00   3rd Qu.: 1.0000   3rd Qu.: 5.000   3rd Qu.: 1.0000  
                                  Max.   :106.00   Max.   :17.0000   Max.   :43.000   Max.   :21.0000  
    ag02Cnt          vb01Prc           nn01Prc          ag01Prc           ag02Prc             ID           
 Min.   :0.0000   Min.   :0.00000   Min.   :0.0000   Min.   :0.00000   Min.   :0.00000   Length:681652     
 1st Qu.:0.0000   1st Qu.:0.00000   1st Qu.:0.1304   1st

# ANOVA (`aov_4`)

| name         | meta   | descr                       |
|--------------|--------|-----------------------------|
| `ID`         | `u1`   | observation unit (tweet_id) |
| `vb01Prc`    | `d1`   | outcome (dependent)         |
| `ag01Prc`    | `d2`   | outcome (dependent)         |
| `CAMP`       | `b1`   | between tweet               |
| `TYPE`       | `b2`   | between tweet               |

In [25]:
OBSERVED <- c("CAMP","TYPE")

In [26]:
OBSERVED <- NULL

## Verbs

In [27]:
FORMULA <- vb01Prc~CAMP*TYPE+(1|ID)

log0 <- logging0(ifName,infoStr="_aov_4_a1vb01Prc",data=df2)
a1vb01Prc <- aov_4(
    formula       <- FORMULA,
    data          <- df2,
    observed      <- OBSERVED,
)
save(a1vb01Prc,file=log0$ofName)
log1 <- logging1(log0$lfName,log0$t0)

logs0: data/t1007/bigDF__df2__001_unstacked.RData__2020-10-13_06:07:18__4cc463667296__aov_4_a1vb01Prc__nrow_681652.RData.LOG.txt
time0: 2020-10-13 06:07:18
nrows: 681652
ncols: 20
vList: data, ifName, infoStr, lfName, ofName, s0, s1, s2, t0
oSave: data/t1007/bigDF__df2__001_unstacked.RData__2020-10-13_06:07:18__4cc463667296__aov_4_a1vb01Prc__nrow_681652.RData



Contrasts set to contr.sum for the following variables: CAMP, TYPE



time1: 2020-10-13 06:08:19
diff1: Time difference of 1.008598 mins



## Dictionary

In [28]:
FORMULA <- ag01Prc~CAMP*TYPE+(1|ID)

log0 <- logging0(ifName,infoStr="_aov_4_a2ag01Prc",data=df2)
a2ag01Prc <- aov_4(
    formula       <- FORMULA,
    data          <- df2,
    observed      <- OBSERVED,
)
save(a2ag01Prc,file=log0$ofName)
log1 <- logging1(log0$lfName,log0$t0)

logs0: data/t1007/bigDF__df2__001_unstacked.RData__2020-10-13_06:08:19__4cc46663b6b2__aov_4_a2ag01Prc__nrow_681652.RData.LOG.txt
time0: 2020-10-13 06:08:19
nrows: 681652
ncols: 20
vList: data, ifName, infoStr, lfName, ofName, s0, s1, s2, t0
oSave: data/t1007/bigDF__df2__001_unstacked.RData__2020-10-13_06:08:19__4cc46663b6b2__aov_4_a2ag01Prc__nrow_681652.RData



Contrasts set to contr.sum for the following variables: CAMP, TYPE



time1: 2020-10-13 06:09:12
diff1: Time difference of 53.69615 secs



# Quick look at AOV results

In [29]:
summary(a1vb01Prc)

,num Df,den Df,MSE,F,ges,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CAMP,1,681648,0.003373904,690.32244046,1.011701e-03,4.531552e-152
TYPE,1,681648,0.003373904,0.02922063,4.286762e-08,8.642707e-01
CAMP:TYPE,1,681648,0.003373904,142.14341243,2.084856e-04,9.115624e-33


In [30]:
summary(a2ag01Prc)

,num Df,den Df,MSE,F,ges,Pr(>F)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CAMP,1,681648,0.003620482,1102.265888,0.0016144496,1.659872e-241
TYPE,1,681648,0.003620482,5.538231,0.0000081247,1.860567e-02
CAMP:TYPE,1,681648,0.003620482,506.682508,0.0007427678,3.673188e-112


## Summarize data by `CAMP` ⨯ `TYPE`

In [31]:
smr0 <- df2 %>%
    group_by(CAMP,TYPE) %>%
    summarise(
        N = n(),
        wcM = mean(wordCnt),
        wcSD = sd(wordCnt),
        wcSE = wcSD / sqrt(N),
        vbM = mean(vb01Prc),
        vbSD = sd(vb01Prc),
        vbSE = vbSD / sqrt(N),
        agM = mean(ag01Prc),
        agSD = sd(ag01Prc),
        agSE = agSD / sqrt(N),
        .groups = "drop"
    ) 
smr0

CAMP,TYPE,N,wcM,wcSD,wcSE,vbM,vbSD,vbSE,agM,agSD,agSE
<fct>,<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
metoo,ctrl,317320,15.43141,7.623185,0.01353280,0.04490575,0.06396794,0.0001135569,0.04017294,0.06624099,1.175921e-04
metoo,orig,296315,20.90304,7.643659,0.01404185,0.04206042,0.05079466,0.0000933128,0.03411315,0.05241299,9.628577e-05
sexstrike,ctrl,36411,17.72418,12.190332,0.06388505,0.04828241,0.06526801,0.0003420457,0.04277857,0.06702385,3.512474e-04
sexstrike,orig,31606,26.21116,14.118213,0.07941361,0.05104731,0.05086050,0.0002860855,0.04769121,0.05567579,3.131711e-04


## Summarize data by `CAMP`

In [32]:
smr0 <- df2 %>%
    group_by(CAMP) %>%
    summarise(
        N = n(),
        wcM = mean(wordCnt),
        wcSD = sd(wordCnt),
        wcSE = wcSD / sqrt(N),
        vbM = mean(vb01Prc),
        vbSD = sd(vb01Prc),
        vbSE = vbSD / sqrt(N),
        agM = mean(ag01Prc),
        agSD = sd(ag01Prc),
        agSE = agSD / sqrt(N),
        .groups = "drop"
    ) 
smr0

CAMP,N,wcM,wcSD,wcSE,vbM,vbSD,vbSE,agM,agSD,agSE
<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
metoo,613635,18.07358,8.108003,0.01035044,0.04353178,0.05799904,7.403988e-05,0.03724676,0.06003949,7.664467e-05
sexstrike,68017,21.66789,13.787225,0.05286499,0.04956720,0.05902798,2.263337e-04,0.04506136,0.06205744,2.379497e-04


## Summarize by type `TYPE`

In [33]:
smr0 <- df2 %>%
    group_by(TYPE) %>%
    summarise(
        N = n(),
        wcM = mean(wordCnt),
        wcSD = sd(wordCnt),
        wcSE = wcSD / sqrt(N),
        vbM = mean(vb01Prc),
        vbSD = sd(vb01Prc),
        vbSE = vbSD / sqrt(N),
        agM = mean(ag01Prc),
        agSD = sd(ag01Prc),
        agSE = agSD / sqrt(N),
        .groups = "drop"
    ) 
smr0

TYPE,N,wcM,wcSD,wcSE,vbM,vbSD,vbSE,agM,agSD,agSE
<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ctrl,353731,15.66741,8.24091,0.01385602,0.04525333,0.06411110,1.077945e-04,0.04044115,0.06632663,1.115196e-04
orig,327921,21.41465,8.62897,0.01506865,0.04292660,0.05087011,8.883377e-05,0.03542185,0.05288819,9.235791e-05


## Generate APA-like LaTeX for AOV

In [34]:
apa0vb <- apa_print(a1vb01Prc)
apa0ag <- apa_print(a2ag01Prc)

In [35]:
apa0vb$table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,CAMP,690.32,1,"681,648",0.00,< .001,.001
2,TYPE,0.03,1,"681,648",0.00,.864,.000
3,CAMP $\times$ TYPE,142.14,1,"681,648",0.00,< .001,.000


In [36]:
display_markdown(apaMd(apa0vb["full_result"]))


### full_result

- full_result for `CAMP` is 

  - $F(1, 681,648) = 690.32$, $\mathit{MSE} = 0.00$, $p < .001$, $\hat{\eta}^2_G = .001$

- full_result for `TYPE` is 

  - $F(1, 681,648) = 0.03$, $\mathit{MSE} = 0.00$, $p = .864$, $\hat{\eta}^2_G = .000$

- full_result for `CAMP_TYPE` is 

  - $F(1, 681,648) = 142.14$, $\mathit{MSE} = 0.00$, $p < .001$, $\hat{\eta}^2_G = .000$


In [37]:
apa0ag$table

,Effect,F,df1,df2,MSE,p,ges
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
1,CAMP,"1,102.27",1,"681,648",0.00,< .001,.002
2,TYPE,5.54,1,"681,648",0.00,.019,.000
3,CAMP $\times$ TYPE,506.68,1,"681,648",0.00,< .001,.001


In [38]:
display_markdown(apaMd(apa0ag["full_result"]))


### full_result

- full_result for `CAMP` is 

  - $F(1, 681,648) = 1,102.27$, $\mathit{MSE} = 0.00$, $p < .001$, $\hat{\eta}^2_G = .002$

- full_result for `TYPE` is 

  - $F(1, 681,648) = 5.54$, $\mathit{MSE} = 0.00$, $p = .019$, $\hat{\eta}^2_G = .000$

- full_result for `CAMP_TYPE` is 

  - $F(1, 681,648) = 506.68$, $\mathit{MSE} = 0.00$, $p < .001$, $\hat{\eta}^2_G = .001$


# Confidence intervals

In [60]:
cohens_d(ag01Prc~CAMP, data = df2)

Cohens_d,CI,CI_low,CI_high
<dbl>,<dbl>,<dbl>,<dbl>
-0.1297162,0.95,-0.1376398,-0.1217924


In [61]:
cohens_d(ag01Prc~TYPE, data = df2)

Cohens_d,CI,CI_low,CI_high
<dbl>,<dbl>,<dbl>,<dbl>
0.08332555,0.95,0.07857221,0.08807883


# Estimated marginal means (Least-squares means)

## Full model

In [39]:
a1vb01PrcEm1 <- emmeans(a1vb01Prc, specs<-c("CAMP", "TYPE"))
a2ag01PrcEm1 <- emmeans(a2ag01Prc, specs<-c("CAMP", "TYPE"))

In [40]:
a1vb01PrcEm1

 CAMP      TYPE  emmean        SE     df lower.CL upper.CL
 metoo     ctrl 0.04491 0.0001031 681648  0.04470  0.04511
 sexstrike ctrl 0.04828 0.0003044 681648  0.04769  0.04888
 metoo     orig 0.04206 0.0001067 681648  0.04185  0.04227
 sexstrike orig 0.05105 0.0003267 681648  0.05041  0.05169

Confidence level used: 0.95 

In [41]:
a2ag01PrcEm1

 CAMP      TYPE emmean       SE     df lower.CL upper.CL
 metoo     ctrl 0.0402 0.000107 681648   0.0400   0.0404
 sexstrike ctrl 0.0428 0.000315 681648   0.0422   0.0434
 metoo     orig 0.0341 0.000111 681648   0.0339   0.0343
 sexstrike orig 0.0477 0.000338 681648   0.0470   0.0484

Confidence level used: 0.95 

In [42]:
pairs(a1vb01PrcEm1)

 contrast                        estimate       SE     df t.ratio p.value
 metoo ctrl - sexstrike ctrl     -0.00338 0.000321 681648 -10.506 <.0001 
 metoo ctrl - metoo orig          0.00285 0.000148 681648  19.175 <.0001 
 metoo ctrl - sexstrike orig     -0.00614 0.000343 681648 -17.926 <.0001 
 sexstrike ctrl - metoo orig      0.00622 0.000323 681648  19.289 <.0001 
 sexstrike ctrl - sexstrike orig -0.00276 0.000447 681648  -6.192 <.0001 
 metoo orig - sexstrike orig     -0.00899 0.000344 681648 -26.147 <.0001 

P value adjustment: tukey method for comparing a family of 4 estimates 

In [43]:
pairs(a2ag01PrcEm1)

 contrast                        estimate       SE     df t.ratio p.value
 metoo ctrl - sexstrike ctrl     -0.00261 0.000333 681648  -7.826 <.0001 
 metoo ctrl - metoo orig          0.00606 0.000154 681648  39.423 <.0001 
 metoo ctrl - sexstrike orig     -0.00752 0.000355 681648 -21.184 <.0001 
 sexstrike ctrl - metoo orig      0.00867 0.000334 681648  25.933 <.0001 
 sexstrike ctrl - sexstrike orig -0.00491 0.000463 681648 -10.620 <.0001 
 metoo orig - sexstrike orig     -0.01358 0.000356 681648 -38.136 <.0001 

P value adjustment: tukey method for comparing a family of 4 estimates 

In [44]:
TEST=adjusted("bonferroni")

In [45]:
TEST=adjusted("free")

In [46]:
summary(as.glht(pairs(a1vb01PrcEm1)),test=TEST)


	 Simultaneous Tests for General Linear Hypotheses

Linear Hypotheses:
                                       Estimate Std. Error t value Pr(>|t|)    
metoo ctrl - sexstrike ctrl == 0     -0.0033767  0.0003214 -10.506  < 2e-16 ***
metoo ctrl - metoo orig == 0          0.0028453  0.0001484  19.175  < 2e-16 ***
metoo ctrl - sexstrike orig == 0     -0.0061416  0.0003426 -17.926  < 2e-16 ***
sexstrike ctrl - metoo orig == 0      0.0062220  0.0003226  19.289  < 2e-16 ***
sexstrike ctrl - sexstrike orig == 0 -0.0027649  0.0004466  -6.192 5.96e-10 ***
metoo orig - sexstrike orig == 0     -0.0089869  0.0003437 -26.147 4.95e-14 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
(Adjusted p values reported -- free method)


In [47]:
summary(as.glht(pairs(a2ag01PrcEm1)),test=TEST)


	 Simultaneous Tests for General Linear Hypotheses

Linear Hypotheses:
                                       Estimate Std. Error t value Pr(>|t|)    
metoo ctrl - sexstrike ctrl == 0     -0.0026056  0.0003329  -7.826 1.82e-14 ***
metoo ctrl - metoo orig == 0          0.0060598  0.0001537  39.423  < 2e-16 ***
metoo ctrl - sexstrike orig == 0     -0.0075183  0.0003549 -21.184  < 2e-16 ***
sexstrike ctrl - metoo orig == 0      0.0086654  0.0003341  25.933  < 2e-16 ***
sexstrike ctrl - sexstrike orig == 0 -0.0049126  0.0004626 -10.620  < 2e-16 ***
metoo orig - sexstrike orig == 0     -0.0135781  0.0003560 -38.136 1.82e-14 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
(Adjusted p values reported -- free method)


## Contrasts

# coef(contrast(a1vb01PrcEm1))

In [48]:
coef(contrast(a2ag01PrcEm1))

,CAMP,TYPE,c.1,c.2,c.3,c.4
,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>
metoo ctrl,metoo,ctrl,0.75,-0.25,-0.25,-0.25
sexstrike ctrl,sexstrike,ctrl,-0.25,0.75,-0.25,-0.25
metoo orig,metoo,orig,-0.25,-0.25,0.75,-0.25
sexstrike orig,sexstrike,orig,-0.25,-0.25,-0.25,0.75


In [49]:
apa0vbEm1 <- apa_print(a1vb01PrcEm1)
apa0agEm1 <- apa_print(a2ag01PrcEm1)

In [50]:
apa0vbEm1$table

,TYPE,CAMP,estimate,ci,statistic,p.value
,<chr>,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
metoo_ctrl,ctrl,metoo,0.04,"$[0.04$, $0.05]$",435.50,< .001
sexstrike_ctrl,,sexstrike,0.05,"$[0.05$, $0.05]$",158.61,< .001
metoo_orig,orig,metoo,0.04,"$[0.04$, $0.04]$",394.17,< .001
sexstrike_orig,,sexstrike,0.05,"$[0.05$, $0.05]$",156.24,< .001


In [51]:
apa0agEm1$table

,TYPE,CAMP,estimate,ci,statistic,p.value
,<chr>,<chr>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
metoo_ctrl,ctrl,metoo,0.04,"$[0.04$, $0.04]$",376.10,< .001
sexstrike_ctrl,,sexstrike,0.04,"$[0.04$, $0.04]$",135.66,< .001
metoo_orig,orig,metoo,0.03,"$[0.03$, $0.03]$",308.61,< .001
sexstrike_orig,,sexstrike,0.05,"$[0.05$, $0.05]$",140.91,< .001


In [52]:
display_markdown(apaMd(apa0vbEm1["full_result"]))


### full_result

- full_result for `metoo_ctrl` is 

  - $M = 0.04$, 95\% CI $[0.04$, $0.05]$, $t(681,648) = 435.50$, $p < .001$

- full_result for `sexstrike_ctrl` is 

  - $M = 0.05$, 95\% CI $[0.05$, $0.05]$, $t(681,648) = 158.61$, $p < .001$

- full_result for `metoo_orig` is 

  - $M = 0.04$, 95\% CI $[0.04$, $0.04]$, $t(681,648) = 394.17$, $p < .001$

- full_result for `sexstrike_orig` is 

  - $M = 0.05$, 95\% CI $[0.05$, $0.05]$, $t(681,648) = 156.24$, $p < .001$


In [53]:
display_markdown(apaMd(apa0agEm1["full_result"]))


### full_result

- full_result for `metoo_ctrl` is 

  - $M = 0.04$, 95\% CI $[0.04$, $0.04]$, $t(681,648) = 376.10$, $p < .001$

- full_result for `sexstrike_ctrl` is 

  - $M = 0.04$, 95\% CI $[0.04$, $0.04]$, $t(681,648) = 135.66$, $p < .001$

- full_result for `metoo_orig` is 

  - $M = 0.03$, 95\% CI $[0.03$, $0.03]$, $t(681,648) = 308.61$, $p < .001$

- full_result for `sexstrike_orig` is 

  - $M = 0.05$, 95\% CI $[0.05$, $0.05]$, $t(681,648) = 140.91$, $p < .001$


In [54]:
apa0vbEm1Pa <- apa_print(pairs(a1vb01PrcEm1))
apa0agEm1Pa <- apa_print(pairs(a2ag01PrcEm1))

In [55]:
apa0vbEm1Pa$table

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
metooctrl_sexstrikectrl,metoo ctrl - sexstrike ctrl,0.00,"$[0.00$, $0.00]$",-10.51,< .001
metooctrl_metooorig,metoo ctrl - metoo orig,0.00,"$[0.00$, $0.00]$",19.18,< .001
metooctrl_sexstrikeorig,metoo ctrl - sexstrike orig,-0.01,"$[-0.01$, $-0.01]$",-17.93,< .001
sexstrikectrl_metooorig,sexstrike ctrl - metoo orig,0.01,"$[0.01$, $0.01]$",19.29,< .001
sexstrikectrl_sexstrikeorig,sexstrike ctrl - sexstrike orig,0.00,"$[0.00$, $0.00]$",-6.19,< .001
metooorig_sexstrikeorig,metoo orig - sexstrike orig,-0.01,"$[-0.01$, $-0.01]$",-26.15,< .001


In [56]:
apa0agEm1Pa$table

,contrast,estimate,ci,statistic,p.value
,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>,<ppj_lbll>
metooctrl_sexstrikectrl,metoo ctrl - sexstrike ctrl,0.00,"$[0.00$, $0.00]$",-7.83,< .001
metooctrl_metooorig,metoo ctrl - metoo orig,0.01,"$[0.01$, $0.01]$",39.42,< .001
metooctrl_sexstrikeorig,metoo ctrl - sexstrike orig,-0.01,"$[-0.01$, $-0.01]$",-21.18,< .001
sexstrikectrl_metooorig,sexstrike ctrl - metoo orig,0.01,"$[0.01$, $0.01]$",25.93,< .001
sexstrikectrl_sexstrikeorig,sexstrike ctrl - sexstrike orig,0.00,"$[-0.01$, $0.00]$",-10.62,< .001
metooorig_sexstrikeorig,metoo orig - sexstrike orig,-0.01,"$[-0.01$, $-0.01]$",-38.14,< .001


In [57]:
display_markdown(apaMd(apa0vbEm1Pa["full_result"]))


### full_result

- full_result for `metooctrl_sexstrikectrl` is 

  - $\Delta M = 0.00$, 95\% CI $[0.00$, $0.00]$, $t(681,648) = -10.51$, $p < .001$

- full_result for `metooctrl_metooorig` is 

  - $\Delta M = 0.00$, 95\% CI $[0.00$, $0.00]$, $t(681,648) = 19.18$, $p < .001$

- full_result for `metooctrl_sexstrikeorig` is 

  - $\Delta M = -0.01$, 95\% CI $[-0.01$, $-0.01]$, $t(681,648) = -17.93$, $p < .001$

- full_result for `sexstrikectrl_metooorig` is 

  - $\Delta M = 0.01$, 95\% CI $[0.01$, $0.01]$, $t(681,648) = 19.29$, $p < .001$

- full_result for `sexstrikectrl_sexstrikeorig` is 

  - $\Delta M = 0.00$, 95\% CI $[0.00$, $0.00]$, $t(681,648) = -6.19$, $p < .001$

- full_result for `metooorig_sexstrikeorig` is 

  - $\Delta M = -0.01$, 95\% CI $[-0.01$, $-0.01]$, $t(681,648) = -26.15$, $p < .001$


In [58]:
display_markdown(apaMd(apa0agEm1Pa["full_result"]))


### full_result

- full_result for `metooctrl_sexstrikectrl` is 

  - $\Delta M = 0.00$, 95\% CI $[0.00$, $0.00]$, $t(681,648) = -7.83$, $p < .001$

- full_result for `metooctrl_metooorig` is 

  - $\Delta M = 0.01$, 95\% CI $[0.01$, $0.01]$, $t(681,648) = 39.42$, $p < .001$

- full_result for `metooctrl_sexstrikeorig` is 

  - $\Delta M = -0.01$, 95\% CI $[-0.01$, $-0.01]$, $t(681,648) = -21.18$, $p < .001$

- full_result for `sexstrikectrl_metooorig` is 

  - $\Delta M = 0.01$, 95\% CI $[0.01$, $0.01]$, $t(681,648) = 25.93$, $p < .001$

- full_result for `sexstrikectrl_sexstrikeorig` is 

  - $\Delta M = 0.00$, 95\% CI $[-0.01$, $0.00]$, $t(681,648) = -10.62$, $p < .001$

- full_result for `metooorig_sexstrikeorig` is 

  - $\Delta M = -0.01$, 95\% CI $[-0.01$, $-0.01]$, $t(681,648) = -38.14$, $p < .001$


In [ ]:
#                      ___           ___                    ___           ___                   
#                     /\  \         /\__\                  /\__\         /\  \         _____    
#        ___          \:\  \       /:/ _/_                /:/ _/_        \:\  \       /::\  \   
#       /\__\          \:\  \     /:/ /\__\              /:/ /\__\        \:\  \     /:/\:\  \  
#      /:/  /      ___ /::\  \   /:/ /:/ _/_            /:/ /:/ _/_   _____\:\  \   /:/  \:\__\ 
#     /:/__/      /\  /:/\:\__\ /:/_/:/ /\__\          /:/_/:/ /\__\ /::::::::\__\ /:/__/ \:|__|
#    /::\  \      \:\/:/  \/__/ \:\/:/ /:/  /          \:\/:/ /:/  / \:\~~\~~\/__/ \:\  \ /:/  /
#   /:/\:\  \      \::/__/       \::/_/:/  /            \::/_/:/  /   \:\  \        \:\  /:/  / 
#   \/__\:\  \      \:\  \        \:\/:/  /              \:\/:/  /     \:\  \        \:\/:/  /  
#        \:\__\      \:\__\        \::/  /                \::/  /       \:\__\        \::/  /   
#         \/__/       \/__/         \/__/                  \/__/         \/__/         \/__/    
#  




# Handy plots

Quote from: https://cran.r-project.org/web/packages/emmeans/vignettes/comparisons.html
> The blue bars are confidence intervals for the EMMs, and the red arrows are for the comparisons among them. If an arrow from one mean overlaps an arrow from another group, the difference is not “significant,” based on the adjust setting (which defaults to "tukey") and the value of alpha (which defaults to 0.05). See the “xplanations” supplement for details on how these are derived.

In [ ]:
library(repr)
options(repr.plot.width=8, repr.plot.height=4)
# fig.height = 10
# fig.width = 5
plot(a1vb01PrcEm2, comparisons=TRUE, xlab="% Verb")+
coord_fixed(ratio = 0.004,xlim=c(0.030, 0.055),ylim=NULL,expand=TRUE,clip="on")
#+ 
#  theme(
#      plot.margin = margin(.2,.2,.2,.2, "cm"),
#      plot.background = element_rect(fill = "darkgrey")
#  )

plot(a2ag01PrcEm2, comparisons=TRUE, xlab="% Dict")+
coord_fixed(ratio = 0.004,xlim=c(0.030, 0.055),ylim=NULL,expand=TRUE,clip="on")


## Effect of TYPE (ctrl, orig) conditional on CAMP (#sexstrike, #metoo)

In [ ]:
a1vb01PrcEm2 <- emmeans(a1vb01Prc, specs<-c("TYPE"), by<-"CAMP")
a2ag01PrcEm2 <- emmeans(a2ag01Prc, specs<-c("TYPE"), by<-"CAMP")

In [ ]:
a1vb01PrcEm2

In [ ]:
a2ag01PrcEm2

In [ ]:
pairs(a1vb01PrcEm2)

In [ ]:
pairs(a2ag01PrcEm2)

In [ ]:
summary(as.glht(pairs(a1vb01PrcEm2)),test=TEST)

In [ ]:
summary(as.glht(pairs(a2ag01PrcEm2)),test=TEST)

# Helpers

In [ ]:
?mixed

In [ ]:
?scale

In [ ]:
?lmer

In [ ]:
?aov_4

In [ ]:
?emmeans

In [ ]:
?coord_fixed

# Extras

In [ ]:
df2$retweets <- as.numeric(df2$retweets) # TODO FIXME This should be fixed elsewhere!

In [ ]:
df2$ReTw <- cut(df2$retweets,
                breaks=c(0, 5, 25, 125, Inf), 
                right  = FALSE,
              # labels=c("low","med","high","viral")
               )

str(df2)

# Playground

In [ ]:
plot_grid(
  afex_plot(
      a1vb01Prc, 
      x="TYPE", 
      trace="CAMP",
      error = "between", 
      data_geom = geom_quasirandom, 
      data_alpha = 0.3) + 
    coord_cartesian(ylim = c(0, 1)),
  afex_plot(
      a2ag01Prc, 
      x="TYPE", 
      trace="CAMP",
      error = "between", 
      data_geom = geom_quasirandom, 
      data_alpha = 0.3) +
    coord_cartesian(ylim = c(0, 1))
)

In [ ]:
qqPlot(a1vb01Prc$aov$residuals)

In [ ]:
qqPlot(a2ag01Prc$aov$residuals)

In [ ]:
shapiro.test(a1vb01Prc$aov$residuals)

In [ ]:
shapiro.test(a2ag01Prc$aov$residuals)

In [ ]:
ad.test(a1vb01Prc$aov$residuals)$p.value

In [ ]:
ad.test(a2ag01Prc$aov$residuals)$p.value

In [ ]:
test_levene(a1vb01Prc)

In [ ]:
test_levene(a2ag01Prc)

In [ ]:
library("pwr")

In [ ]:
?pwr.anova.test

In [ ]:
?summary

In [ ]:
convert_ipynb

In [ ]:
?apa_print


In [ ]:
methods("apa_print")